In [1]:
import os
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision

from torch.utils.data import Dataset, DataLoader, BatchSampler, random_split
from torchvision import transforms
from PIL import Image

In [2]:
from torchvision import models
backbone = models.resnet18(weights='IMAGENET1K_V1')

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 57.6MB/s]


In [3]:
backbone

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [4]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/NNDL_Project

Mounted at /content/drive
/content/drive/MyDrive/NNDL_Project


In [5]:
%%capture
! unzip -o train_shuffle.zip

In [6]:
%%capture
! unzip -o test_shuffle.zip

In [7]:
# Create Dataset class for multilabel classification
class MultiClassImageDataset(Dataset):
    def __init__(self, ann_df, super_map_df, img_dir, transform=None):
        self.ann_df = ann_df
        self.super_map_df = super_map_df
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.ann_df)

    def __getitem__(self, idx):
        img_name = self.ann_df['image'][idx]
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')

        super_idx = self.ann_df['superclass_index'][idx]
        super_label = self.super_map_df['class'][super_idx]

        if self.transform:
            image = self.transform(image)

        return image, super_idx, super_label

class MultiClassImageTestDataset(Dataset):
    def __init__(self, super_map_df, img_dir, transform=None):
        self.super_map_df = super_map_df
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self): # Count files in img_dir
        return len([fname for fname in os.listdir(self.img_dir)])

    def __getitem__(self, idx):
        img_name = str(idx) + '.jpg'
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, img_name

In [8]:
train_ann_df = pd.read_csv('train_data.csv')
super_map_df = pd.read_csv('superclass_mapping.csv')
sub_map_df = pd.read_csv('subclass_mapping.csv')

train_img_dir = 'train_shuffle'
test_img_dir = 'test_shuffle'

image_preprocessing = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0), std=(1)),
])

# Create train and val split
train_dataset = MultiClassImageDataset(train_ann_df, super_map_df, train_img_dir, transform=image_preprocessing)
train_dataset, val_dataset = random_split(train_dataset, [0.9, 0.1])

# Create test dataset
test_dataset = MultiClassImageTestDataset(super_map_df, test_img_dir, transform=image_preprocessing)

# Create dataloaders
batch_size = 64
train_loader = DataLoader(train_dataset,
                          batch_size=batch_size,
                          shuffle=True, drop_last = True)

val_loader = DataLoader(val_dataset,
                        batch_size=batch_size,
                        shuffle=True, drop_last = True)

test_loader = DataLoader(test_dataset,
                         batch_size=1,
                         shuffle=False)

In [17]:
# Simple CNN
class CNN(nn.Module):
    def __init__(self, backbone, finetune=False):
        super().__init__()

        self.backbone = backbone

        if not finetune:
          for param in self.backbone.parameters():
            param.requires_grad = False
        else:
          for i, layer in enumerate(self.backbone.children()):
            if i < 6: #just train last of the main convolutional layers
              for param in layer.parameters():
                param.requires_grad = False
            else:
              for param in layer.parameters():
                param.requires_grad = True

        num_ftrs = self.backbone.fc.in_features
        self.backbone.fc = nn.Linear(num_ftrs, 4)

    def forward(self, x):
        super_out = self.backbone(x)
        return super_out

class Trainer():
    def __init__(self, model, criterion, optimizer, train_loader, val_loader, test_loader=None, device='cuda'):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.test_loader = test_loader
        self.device = device

    def train_epoch(self):
        running_loss = 0.0
        device = self.device
        for i, data in enumerate(self.train_loader):
            inputs, super_labels = data[0].to(device), data[1].to(device)

            self.optimizer.zero_grad()
            super_outputs = self.model(inputs)
            loss = self.criterion(super_outputs, super_labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Training loss: {running_loss/i:.3f}')

    def validate_epoch(self):
        super_correct = 0
        total = 0
        running_loss = 0.0
        device = self.device
        with torch.no_grad():
            for i, data in enumerate(self.val_loader):
                inputs, super_labels = data[0].to(device), data[1].to(device)

                super_outputs = self.model(inputs)
                loss = self.criterion(super_outputs, super_labels)
                _, super_predicted = torch.max(super_outputs.data, 1)

                total += super_labels.size(0)
                super_correct += (super_predicted == super_labels).sum().item()
                running_loss += loss.item()

        print(f'Validation loss: {running_loss/i:.3f}')
        print(f'Validation superclass acc: {100 * super_correct / total:.2f} %')

    def test(self, save_to_csv=False, return_predictions=False):
        if not self.test_loader:
            raise NotImplementedError('test_loader not specified')

        # Evaluate on test set, in this simple demo no special care is taken for novel/unseen classes
        test_predictions = {'image': [], 'superclass_index': []}
        with torch.no_grad():
            for i, data in enumerate(self.test_loader):
                inputs, img_name = data[0].to(device), data[1]

                super_outputs = self.model(inputs)
                _, super_predicted = torch.max(super_outputs.data, 1)

                test_predictions['image'].append(img_name[0])
                test_predictions['superclass_index'].append(super_predicted.item())

        test_predictions = pd.DataFrame(data=test_predictions)

        if save_to_csv:
            test_predictions.to_csv('transfer_learning_superclass_test_predictions.csv', index=False)

        if return_predictions:
            return test_predictions

In [18]:
def fine_tune(backbone):
  for i, layer in enumerate(backbone.children()):
    if i < 6: #just train last of the main convolutional layers
      for param in layer.parameters():
        param.requires_grad = False
    else:
      for param in layer.parameters():
        param.requires_grad = True
  return

In [19]:
# Init model and trainer
device = 'cuda'
backbone = models.resnet18(weights='IMAGENET1K_V1')
finetune = False
model = CNN(backbone, finetune=finetune).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
trainer = Trainer(model, criterion, optimizer, train_loader, val_loader, test_loader)

In [20]:
# Training loop
print('Trainable parameters: ', sum(p.numel() for p in model.parameters() if p.requires_grad))
print('Total parameters: ', sum(p.numel() for p in model.parameters()))
print('----------------------')
print('')
for epoch in range(25):
    print(f'Epoch {epoch+1}')
    trainer.train_epoch()
    trainer.validate_epoch()
    print('')
    if epoch == 10:
      fine_tune(trainer.model.backbone)
      print('')
      print('Trainable parameters: ', sum(p.numel() for p in trainer.model.parameters() if p.requires_grad))
      print('Total parameters: ', sum(p.numel() for p in trainer.model.parameters()))
      print('----------------------')
      print('')
      trainer = Trainer(trainer.model, trainer.criterion, trainer.optimizer, trainer.train_loader, trainer.val_loader, trainer.test_loader)


print('Finished Training')

Trainable parameters:  2052
Total parameters:  11178564
----------------------

Epoch 1
Training loss: 0.881
Validation loss: 0.790
Validation superclass acc: 69.97 %

Epoch 2
Training loss: 0.667
Validation loss: 0.788
Validation superclass acc: 70.49 %

Epoch 3
Training loss: 0.608
Validation loss: 0.738
Validation superclass acc: 71.53 %

Epoch 4
Training loss: 0.590
Validation loss: 0.725
Validation superclass acc: 72.74 %

Epoch 5
Training loss: 0.583
Validation loss: 0.668
Validation superclass acc: 75.35 %

Epoch 6
Training loss: 0.580
Validation loss: 0.708
Validation superclass acc: 72.57 %

Epoch 7
Training loss: 0.560
Validation loss: 0.698
Validation superclass acc: 74.83 %

Epoch 8
Training loss: 0.556
Validation loss: 0.699
Validation superclass acc: 72.22 %

Epoch 9
Training loss: 0.561
Validation loss: 0.662
Validation superclass acc: 73.78 %

Epoch 10
Training loss: 0.552
Validation loss: 0.676
Validation superclass acc: 76.39 %

Epoch 11
Training loss: 0.550
Validatio

KeyboardInterrupt: ignored

In [ ]:
trainer.model.eval()
trainer.test(save_to_csv=False, return_predictions=True)

'''
This simple baseline scores the following test accuracy

Superclass Accuracy
Overall: 43.83 %
Seen: 61.11 %
Unseen: 0.00 %

Subclass Accuracy
Overall: 2.03 %
Seen: 9.56 %
Unseen: 0.00 %
'''